<a href="https://colab.research.google.com/github/ougrid/my-knowledge-resource/blob/master/Copy_of_BKK_I_O_Extended_Gemini_Structured_Decision_Making.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install --upgrade --user google-cloud-aiplatform

In [ ]:
# Authenticate user to Google Cloud
from google.colab import auth

auth.authenticate_user()

In [ ]:
# Define project information
PROJECT_ID = "your_project_id"  # @param {type:"string"}
LOCATION = "asia-southeast1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

## Utils

In [ ]:
import re
import json

def decode_thai_unicode(input_string):
    # Regular expression to find Unicode escape sequences
    pattern = r'\\([0-7]{3})'

    # Function to convert octal to character
    def octal_to_char(match):
        return chr(int(match.group(1), 8))

    # Replace all Unicode escape sequences with their corresponding characters
    return re.sub(pattern, octal_to_char, input_string)


## Function Calling

In [ ]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

model = GenerativeModel("gemini-1.5-pro-001")

get_exchange_rate_func = FunctionDeclaration(
    name="get_exchange_rate",
    description="Get the exchange rate for currencies between countries",
    parameters={
    "type": "object",
    "properties": {
        "currency_date": {
            "type": "string",
            "description": "A date that must always be in YYYY-MM-DD format or the value 'latest' if a time period is not specified"
        },
        "currency_from": {
            "type": "string",
            "description": "The currency to convert from in ISO 4217 format"
        },
        "currency_to": {
            "type": "string",
            "description": "The currency to convert to in ISO 4217 format"
        }
    },
         "required": [
            "currency_from",
            "currency_date",
      ]
  },
)

exchange_rate_tool = Tool(
    function_declarations=[get_exchange_rate_func],
)

prompt = """What is the exchange rate from Singapore dollars to Thai Baht?
How much is 100 Singapore dollars worth in Thai Baht?"""

response = model.generate_content(
    prompt,
    tools=[exchange_rate_tool],
)

response.candidates[0].content

params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key[9:]] = value
params



{'date': 'latest', 'from': 'SGD', 'to': 'THB'}

In [ ]:
import requests
url = f"https://api.frankfurter.app/{params['date']}"
api_response = requests.get(url, params=params)
api_response.text

response = model.generate_content(
    [
    Content(role="user", parts=[
        Part.from_text(prompt + """Give your answer in steps with lots of detail
            and context, including the exchange rate and date."""),
    ]),
    Content(role="function", parts=[
        Part.from_dict({
            "function_call": {
                "name": "get_exchange_rate",
            }
        })
    ]),
    Content(role="function", parts=[
        Part.from_function_response(
            name="get_exchange_rate",
            response={
                "content": api_response.text,
            }
        )
    ]),
    ],
    tools=[exchange_rate_tool],
)


response.candidates[0].content.parts[0].text

"Today's date is 2024-07-12. The exchange rate from Singapore Dollars (SGD) to Thai Baht (THB) is 1 SGD = 26.921 THB. Therefore, 100 Singapore Dollars is equal to 2692.1 Thai Baht (100 x 26.921 = 2692.1). \n"

## Stuctured Response

In [ ]:
from vertexai.generative_models import (
    FunctionDeclaration,
    GenerativeModel,
    GenerationConfig,
    Tool,
)

In [ ]:
model = GenerativeModel(
    "gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0),
)

In [ ]:
get_summary_params = FunctionDeclaration(
    name="get_summaries",
    description="Get a summary in both English and Thai",
    parameters={
        "type": "object",
        "properties": {
            "english_summary": {
                "type": "string",
                "description": "summary of the content sent, in English",
            },
            "thai_summary": {
                "type": "string",
                "description": "summary of the content sent, in Thai"
            },
            "input_languge": {
                "type": "string",
                "description": "The original language that the input was in",
            },
            "similarity_score": {
                "type": "number",
                "description": "a score between 0-1 for how similar the english summary and Thai summary are",
            },
        },
        "required": [
                        "english_summary",
                        "thai_summary",
                        "input_languge",
                        "similarity_score"
                    ],
    },
)

destination_tool = Tool(
    function_declarations=[get_summary_params],
)

In [ ]:
INPUT_ARTICLE = """A Castro Valley resident was charged Thursday for allegedly slashing the tires of 17 Waymo robotaxis in San Francisco between June 24 and June 26, according to the city’s district attorney.

Prosecutors say the tire slashings were captured by cameras installed on the exterior of Waymo’s robotaxis. This is the latest incident of Waymo vandalism in the Bay Area, where some residents have expressed frustration with the autonomous vehicles.

San Francisco has a history of revolting against robotaxis more broadly. In February, a crowd of people in the city’s Chinatown neighborhood committed another act of robotaxi vandalism by setting a Waymo vehicle on fire. Roughly a year ago, other robotaxi haters placed traffic cones on top of autonomous vehicles, a hack that disabled the vehicles.

This time around, Waymo’s technology seems to have caught one of these alleged crimes on video. In an email to TechCrunch, Waymo said the San Francisco Police Department reviewed footage from the Waymo vehicles’ exterior cameras in order to identify the suspect.

“We can confirm that charges have been brought against the individual who aggressively vandalized a number of Waymo vehicles, some with riders present,” said Waymo spokesperson Katherine Barna in an email. “Waymo is also taking steps to recover the damages sustained, and mitigate the potential for future events.”"""

In [ ]:
response = model.generate_content(
    INPUT_ARTICLE,
    tools=[destination_tool],
)

response.candidates[0].function_calls

[name: "get_summaries"
 args {
   fields {
     key: "english_summary"
     value {
       string_value: "A Castro Valley resident was charged with slashing the tires of 17 Waymo robotaxis in San Francisco. The incidents, which took place between June 24 and June 26, were captured by cameras installed on the vehicles. This is not the first time Waymo robotaxis have been vandalized in the Bay Area, as some residents have expressed frustration with the autonomous vehicles."
     }
   }
   fields {
     key: "input_languge"
     value {
       string_value: "English"
     }
   }
   fields {
     key: "similarity_score"
     value {
       number_value: 0.9
     }
   }
   fields {
     key: "thai_summary"
     value {
       string_value: "\340\270\212\340\270\262\340\270\247\340\271\200\340\270\241\340\270\267\340\270\255\340\270\207\340\270\204\340\270\262\340\270\252\340\271\202\340\270\225\340\270\243\340\271\201\340\270\247\340\270\245\340\270\245\340\270\265\340\270\242\340\271\214\3

In [ ]:
response.candidates[0].function_calls[0].args['english_summary']

'A Castro Valley resident was charged with slashing the tires of 17 Waymo robotaxis in San Francisco. The incidents, which took place between June 24 and June 26, were captured by cameras installed on the vehicles. This is not the first time Waymo robotaxis have been vandalized in the Bay Area, as some residents have expressed frustration with the autonomous vehicles.'

In [ ]:
response.candidates[0].function_calls[0].args['thai_summary']

'ชาวเมืองคาสโตรแวลลีย์ถูกตั้งข้อหาในวันพฤหัสบดีจากข้อกล่าวหาแทงยางรถแท็กซี่ไร้คนขับของ Waymo จำนวน 17 คันในซานฟรานซิสโก เหตุการณ์ดังกล่าวเกิดขึ้นระหว่างวันที่ 24 มิถุนายน ถึง 26 มิถุนายน ซึ่งกล้องที่ติดตั้งบนยานพาหนะสามารถบันทึกภาพไว้ได้ นี่ไม่ใช่ครั้งแรกที่รถแท็กซี่ไร้คนขับของ Waymo ถูกทุบทำลายในพื้นที่อ่าว เนื่องจากผู้อยู่อาศัยบางคนแสดงความไม่พอใจต่อรถยนต์ไร้คนขับ'

In [ ]:
decode_thai_unicode(response.candidates[0].function_calls[0].args['thai_summary'])

'ชาวเมืองคาสโตรแวลลีย์ถูกตั้งข้อหาในวันพฤหัสบดีจากข้อกล่าวหาแทงยางรถแท็กซี่ไร้คนขับของ Waymo จำนวน 17 คันในซานฟรานซิสโก เหตุการณ์ดังกล่าวเกิดขึ้นระหว่างวันที่ 24 มิถุนายน ถึง 26 มิถุนายน ซึ่งกล้องที่ติดตั้งบนยานพาหนะสามารถบันทึกภาพไว้ได้ นี่ไม่ใช่ครั้งแรกที่รถแท็กซี่ไร้คนขับของ Waymo ถูกทุบทำลายในพื้นที่อ่าว เนื่องจากผู้อยู่อาศัยบางคนแสดงความไม่พอใจต่อรถยนต์ไร้คนขับ'

In [ ]:
response.candidates[0].function_calls[0].args['input_languge']

'English'

In [ ]:
response.candidates[0].function_calls[0].args['similarity_score']

0.9

## lets put it in a function

In [ ]:
def process_article(article, extra_instructions=""):
    response = model.generate_content(
        extra_instructions + article,
        tools=[destination_tool],
    )
    english_summary = response.candidates[0].function_calls[0].args['english_summary']
    thai_summary = decode_thai_unicode(response.candidates[0].function_calls[0].args['thai_summary'])
    input_languge = response.candidates[0].function_calls[0].args['input_languge']
    similarity_score = response.candidates[0].function_calls[0].args['similarity_score']
    processed_article = {
        "english_summary": english_summary,
        "thai_summary": thai_summary,
        "input_languge": input_languge,
        "similarity_score": similarity_score,
    }
    return processed_article

In [ ]:
ARTICLE = """In December, we launched our first natively multimodal model Gemini 1.0 in three sizes: Ultra, Pro and Nano. Just a few months later we released 1.5 Pro, with enhanced performance and a breakthrough long context window of 1 million tokens.

Developers and enterprise customers have been putting 1.5 Pro to use in incredible ways and finding its long context window, multimodal reasoning capabilities and impressive overall performance incredibly useful.

We know from user feedback that some applications need lower latency and a lower cost to serve. This inspired us to keep innovating, so today, we’re introducing Gemini 1.5 Flash: a model that’s lighter-weight than 1.5 Pro, and designed to be fast and efficient to serve at scale.

Both 1.5 Pro and 1.5 Flash are available in public preview with a 1 million token context window in Google AI Studio and Vertex AI. And now, 1.5 Pro is also available with a 2 million token context window via waitlist to developers using the API and to Google Cloud customers.

We’re also introducing updates across the Gemini family of models, announcing our next generation of open models, Gemma 2, and sharing progress on the future of AI assistants, with Project Astra.

Gemini family of model updates
The new 1.5 Flash, optimized for speed and efficiency
1.5 Flash is the newest addition to the Gemini model family and the fastest Gemini model served in the API. It’s optimized for high-volume, high-frequency tasks at scale, is more cost-efficient to serve and features our breakthrough long context window.

While it’s a lighter weight model than 1.5 Pro, it’s highly capable of multimodal reasoning across vast amounts of information and delivers impressive quality for its size.

1.5 Flash excels at summarization, chat applications, image and video captioning, data extraction from long documents and tables, and more. This is because it’s been trained by 1.5 Pro through a process called “distillation,” where the most essential knowledge and skills from a larger model are transferred to a smaller, more efficient model.

Read more about 1.5 Flash in our updated Gemini 1.5 technical report, on the Gemini technology page, and learn about 1.5 Flash’s availability and pricing.

Significantly improving 1.5 Pro
Over the last few months, we’ve significantly improved 1.5 Pro, our best model for general performance across a wide range of tasks.

Beyond extending its context window to 2 million tokens, we’ve enhanced its code generation, logical reasoning and planning, multi-turn conversation, and audio and image understanding through data and algorithmic advances. We see strong improvements on public and internal benchmarks for each of these tasks.

1.5 Pro can now follow increasingly complex and nuanced instructions, including ones that specify product-level behavior involving role, format and style. We’ve improved control over the model’s responses for specific use cases, like crafting the persona and response style of a chat agent or automating workflows through multiple function calls. And we’ve enabled users to steer model behavior by setting system instructions.

We added audio understanding in the Gemini API and Google AI Studio, so 1.5 Pro can now reason across image and audio for videos uploaded in Google AI Studio. And we’re now integrating 1.5 Pro into Google products, including Gemini Advanced and in Workspace apps.

Read more about 1.5 Pro in our updated Gemini 1.5 technical report and on the Gemini technology page.

Gemini Nano understands multimodal inputs
Gemini Nano is expanding beyond text-only inputs to include images as well. Starting with Pixel, applications using Gemini Nano with Multimodality will be able to understand the world the way people do — not just through text, but also through sight, sound and spoken language.

Read more about Gemini 1.0 Nano on Android.

Next generation of open models
Today, we’re also sharing a series of updates to Gemma, our family of open models built from the same research and technology used to create the Gemini models.

We’re announcing Gemma 2, our next generation of open models for responsible AI innovation. Gemma 2 has a new architecture designed for breakthrough performance and efficiency, and will be available in new sizes.

The Gemma family is also expanding with PaliGemma, our first vision-language model inspired by PaLI-3. And we’ve upgraded our Responsible Generative AI Toolkit with LLM Comparator for evaluating the quality of model responses."""

In [ ]:
processed_results = process_article(ARTICLE)

processed_results

{'english_summary': 'Gemini 1.5 Flash is a new, lighter-weight model that is faster and more efficient than 1.5 Pro. 1.5 Pro has also been upgraded with a 2 million token context window and improved capabilities. The Gemini family of models is expanding with the addition of Gemini Nano with Multimodality, which can understand both text and images. Gemma 2, the next generation of open models, will be available soon with a new architecture and new sizes.',
 'thai_summary': 'Gemini 1.5 Flash เป็นโมเดลรุ่นใหม่ที่เบากว่า เร็วกว่า และมีประสิทธิภาพมากกว่า 1.5 Pro โมเดล 1.5 Pro ได้รับการอัปเกรดด้วยหน้าต่างบริบท 2 ล้านโทเค็นและความสามารถที่ได้รับการปรับปรุง กลุ่มโมเดล Gemini กำลังขยายตัวด้วยการเพิ่ม Gemini Nano ที่มีความสามารถหลายรูปแบบ ซึ่งสามารถเข้าใจทั้งข้อความและรูปภาพ Gemma 2 ซึ่งเป็นโมเดลโอเพ่นซอร์สรุ่นต่อไป จะพร้อมใช้งานในเร็วๆ นี้ด้วยสถาปัตยกรรมใหม่และขนาดใหม่',
 'input_languge': 'English',
 'similarity_score': 0.9}

## Reflection

Normally would use a prompt to ask it to evaluate

In [ ]:
while processed_results['similarity_score'] < 0.8:
    print("Need to update the similarity")
    processed_results = process_article(ARTICLE, "You need to make the similarity score higher")
else:
    print("Similarity is good\n")
    print("-"*50 + "\n")
    print(processed_results['english_summary'])
    print("-"*50 + "\n")
    print(processed_results['thai_summary'])

Similarity is good

--------------------------------------------------

Gemini 1.5 Flash is a new, lighter-weight model that is faster and more efficient than 1.5 Pro. 1.5 Pro has also been upgraded with a 2 million token context window and improved capabilities. The Gemini family of models is expanding with the addition of Gemini Nano with Multimodality, which can understand both text and images. Gemma 2, the next generation of open models, will be available soon with a new architecture and new sizes.
--------------------------------------------------

Gemini 1.5 Flash เป็นโมเดลรุ่นใหม่ที่เบากว่า เร็วกว่า และมีประสิทธิภาพมากกว่า 1.5 Pro โมเดล 1.5 Pro ได้รับการอัปเกรดด้วยหน้าต่างบริบท 2 ล้านโทเค็นและความสามารถที่ได้รับการปรับปรุง กลุ่มโมเดล Gemini กำลังขยายตัวด้วยการเพิ่ม Gemini Nano ที่มีความสามารถหลายรูปแบบ ซึ่งสามารถเข้าใจทั้งข้อความและรูปภาพ Gemma 2 ซึ่งเป็นโมเดลโอเพ่นซอร์สรุ่นต่อไป จะพร้อมใช้งานในเร็วๆ นี้ด้วยสถาปัตยกรรมใหม่และขนาดใหม่


## From Google Translate

Gemini 1.5 Flash is the next generation model that is lighter, faster, and more powerful than 1.5 Pro. The 1.5 Pro model has been upgraded with a context window of 2 million tokens and improved capabilities. The Gemini model family is expanding with Added Gemini Nano with multiple abilities. It can understand both text and images. Gemma 2 is the next generation open source model. Will be ready to use soon. This with a new architecture and new size.

## Testing with Thai

In [ ]:
ARTICLE = """ลลิษา มโนบาล[2][3] ชื่อเดิม ปราณปรียา มโนบาล[4] (เกิด 27 มีนาคม พ.ศ. 2540) ชื่อในการแสดงว่า ลิซ่า (ฮันกึล: 리사, อังกฤษ: Lisa) เป็นแร็ปเปอร์ นักร้อง นักเต้น และนักแสดงชาวไทย เธอเป็นสมาชิกของแบล็กพิงก์ เกิร์ลกรุปเกาหลีใต้สังกัดวายจีเอนเตอร์เทนเมนต์[5]

ลิซ่าเปิดตัวเป็นศิลปินเดี่ยวกับซิงเกิลอัลบั้มชื่อ ลลิษา ในเดือนกันยายน 2564 อัลบั้มเปิดตัวที่อันดับหนึ่งบนแกออนอัลบั้มชาร์ต และขายได้มากกว่า 7 แสนชุดในสัปดาห์แรก ทำให้เธอเป็นศิลปินหญิงคนแรกที่ทำได้[6] มิวสิกวิดีโอเพลงชื่อเดียวกับอัลบั้ม มีผู้ชมรวม 73.6 ล้านครั้งในวันแรกบนยูทูบ กลายเป็นมิวสิกวิดีโอที่มีผู้ชมมากที่สุดใน 24 ชั่วโมงแรกบนยูทูบโดยศิลปินเดี่ยว[7] ซิงเกิลแรกและซิงเกิลที่สองของอัลบั้ม "ลลิษา" และ "มันนี" ขึ้นถึงสิบอันดับแรกบน บิลบอร์ด โกลบอล 200 โดยเพลงหลังกลายเป็นเพลงแรกของศิลปินเดี่ยวหญิงเคป็อปที่อยู่บนชาร์ต บิลบอร์ด ฮอต 100 และชาร์ตซิงเกิลแห่งสหราชอาณาจักร ได้นานหลายสัปดาห์ นอกจากนี้ ลลิษา และ "มันนี" ยังกลายเป็นอัลบั้มและเพลงแรกของศิลปินเดี่ยวเคป็อปที่มียอดสตรีมถึง 1 พันล้านครั้งบนสปอติฟายด้วย

ลิซ่าได้รับรางวัลมากมาย รวมถึงรางวัลมามาอะวอดส์, เซอร์เคิลชาร์ตมิวสิกอะวอดส์, เอ็มทีวีวิดีโอมิวสิกอะวอดส์ และเอ็มทีวียุโรปมิวสิกอะวอดส์ สองรางวัลหลังเธอเป็นศิลปินเดี่ยวเคป็อปคนแรกที่ได้รับรางวัลดังกล่าว เธอได้บันทึกในบันทึกสถิติโลกกินเนสส์ 8 หัวข้อ มากกว่าศิลปินเดี่ยวเคป็อปคนใด ๆ ทั้งนี้ เธอยังเป็นศิลปินเคป็อปที่มียอดผู้ติดตามมากที่สุดบนอินสตาแกรมเช่นเดียวกับศิลปินเดี่ยวเคป็อปที่มียอดผู้ติดตามมากที่สุดบนยูทูบ

ชีวิตช่วงแรก
ลิซ่าเกิดเมื่อวันที่ 27 มีนาคม พ.ศ. 2540[8] ที่จังหวัดบุรีรัมย์[9] มีชื่อเกิดว่า "ปราณปรียา" แต่ภายหลังเปลี่ยนเป็น "ลลิษา"[10] จากการทายทักของหมอดู มีความหมายว่า "ผู้ที่ได้รับการยกย่อง"[11][12] และมีชื่อเล่นในวัยเด็กว่า "ป๊อกแป๊ก", "ลลิส" และ "ลิส" ตามลำดับ[13] ลิซ่าอาศัยอยู่กับมารดาชื่อ จิตทิพย์ เป็นชาวจังหวัดบุรีรัมย์[14] และพ่อเลี้ยงชื่อ มาร์โค บรุช์ไวเลอร์ (Marco Brüschweiler) ชาวสวิส เป็นที่ปรึกษาการประกอบธุรกิจร้านอาหารและผู้เชี่ยวชาญทางด้านอาหารไทย[15][16] ลิซ่าสำเร็จการศึกษาจากโรงเรียนประภามนตรี 1 และ 2[17]

เมื่ออายุสี่ขวบ ลิซ่าเริ่มหัดเต้นและลงแข่งขันในเวทีต่าง ๆ รวมถึงโครงการทูบีนัมเบอร์วัน[13] และได้เข้าร่วมทีมนักเต้น วีซาคูล (We Zaa Cool) ซึ่งต่อมาในเดือนกันยายน 2552 ทีมได้เข้าแข่งขันในรายการ แอลจีเอนเตอร์เทนเนอร์ ล้านฝันสนั่นโลก ทางช่อง 9 และได้รางวัลพิเศษประเภททีม[18] นอกจากนี้ ลิซ่ายังเป็นตัวแทนของโรงเรียนไปประกวดร้องเพลงในโครงการ 3 คุณธรรมนำไทย ซึ่งจัดโดยศูนย์คุณธรรม และได้รับรางวัลรองชนะเลิศ[17]

ครั้นเมื่อเข้าสู่วัยรุ่น ลิซ่าเริ่มสนใจอุตสาหกรรมเคป็อปและอยากเป็นศิลปินเกาหลี[19] เธอยังแสดงความชื่นชมต่อบิกแบงและทูเอนีวัน และปรารถนาว่าตนอยากประสบความสำเร็จเหมือนกับวงดังกล่าว[20] ในปี 2553 วายจีเอนเตอร์เทนเมนต์มาเปิดออดิชันในประเทศไทยเป็นครั้งแรก จากผู้สมัครราวสี่พันคน ลิซ่าเป็นคนเดียวที่ผ่านการคัดเลือกและได้เข้ามาเป็นศิลปินฝึกหัดของวายจีเมื่อวันที่ 11 เมษายน 2554[21] ขณะออดิชัน เธอยังสร้างความประทับใจให้กับกรรมการชื่อ แดนนี อิม แห่งวงวันไทม์ ซึ่งภายหลังเขาออกมาชื่นชมถึงความมั่นใจบนเวทีและทัศนคติหลังเวทีของเธอ[22]"""

In [ ]:
processed_results = process_article(ARTICLE)

processed_results

{'english_summary': 'Lalisa Manoban, better known by her stage name Lisa, is a Thai rapper, singer, and dancer based in South Korea. She is a member of the South Korean girl group Blackpink under YG Entertainment.',
 'thai_summary': 'ลลิษา มโนบาล หรือที่รู้จักกันในชื่อ ลิซ่า เป็นแร็ปเปอร์ นักร้อง และนักเต้นชาวไทย เธอเป็นสมาชิกของวงเกิร์ลกรุ๊ปเกาหลีใต้ แบล็กพิงก์ ภายใต้สังกัดวายจีเอนเตอร์เทนเมนต์',
 'input_languge': 'Thai',
 'similarity_score': 0.9}